# Query Nested Video Embeddings Index

This notebook demonstrates how to query the nested OpenSearch index with vector search across different embedding modalities (visual-text, visual-image, audio).

## Setup and Configuration

In [1]:
import json
import os
import numpy as np
from typing import List, Dict, Optional
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
from dotenv import load_dotenv
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Initialize OpenSearch Client

In [2]:
# Get AWS credentials and OpenSearch host from environment
opensearch_host = os.getenv('OPENSEARCH_CLUSTER_HOST').replace("https://", "")
aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.getenv('AWS_SESSION_TOKEN')
aws_region = os.getenv('AWS_REGION', 'us-east-1')

# Create AWS credentials
credentials = boto3.Session(
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=aws_region
).get_credentials()

# Initialize OpenSearch client
opensearch_client = OpenSearch(
    hosts=[{'host': opensearch_host, 'port': 443}],
    http_auth=AWSV4SignerAuth(credentials, aws_region),
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_connections=10,
    pool_maxsize=10
)

# Test connection
try:
    info = opensearch_client.info()
    print(f"✓ Connected to OpenSearch: {info['version']['number']}")
except Exception as e:
    print(f"✗ Failed to connect: {e}")

2025-11-12 18:09:58,276 - INFO - GET https://search-condenast-aos-domain-3hmon7me6ct3p5e46snecxe6f4.us-east-1.es.amazonaws.com:443/ [status:200 request:2.630s]


✓ Connected to OpenSearch: 3.1.0


## Define Query Function

In [19]:
def query_nested_embeddings(
    query_vector: List[float],
    k: int = 5,
    index_name: str = 'video_clips_nested'
) -> Dict:
    """
    Query the nested embeddings index using vector similarity search
    
    Args:
        query_vector: 1024D embedding vector to search for
        k: Number of video results to return
        index_name: Name of the OpenSearch index
    
    Returns:
        Dictionary containing search results
    """
    
    query_body = {
        "query": {
            "nested": {
                "path": "embeddings",
                "query": {
                    "knn": {
                        "embeddings.embedding": {
                            "vector": query_vector,
                            "k": 100,  # Increased to get more nested hits per video
                            "filter":
                            {
                                "term": {
                                    "embeddings.embeddingOption": "visual-text"
                                }
                            }
                        }
                    }
                },
                "inner_hits": {
                    "_source": False,
                    "size": 100,  # Get all matching segments per video
                    "fields": [
                        "embeddings.startSec",
                        "embeddings.endSec",
                        "embeddings.embeddingOption",
                    ],
                },
            }
        },
        "size": k,  # Number of videos to return
        "_source": {"excludes": ["embeddings.embedding"]},
    }
    
    try:
        search_results = opensearch_client.search(body=query_body, index=index_name)
        return search_results
    except Exception as e:
        logger.error(f"Error querying index: {e}")
        return {}

print("✓ Query function defined")

✓ Query function defined


## Generate Embeddings from User Query

In [4]:
# Initialize Bedrock client for embedding generation
bedrock_runtime = boto3.client('bedrock-runtime', region_name=aws_region)
BEDROCK_MODEL_ID = "us.twelvelabs.marengo-embed-2-7-v1:0"

def generate_text_embedding(text: str) -> List[float]:
    """
    Generate embedding for text query using Bedrock Marengo model
    
    Args:
        text: Text query to embed
    
    Returns:
        List of floats representing the 1024D embedding
    """
    try:
        print(f"📝 Generating embedding for query: '{text}'")
        
        request_body = {
            "inputType": "text",
            "inputText": text,
            "textTruncate": "none"
        }
        
        response = bedrock_runtime.invoke_model(
            modelId=BEDROCK_MODEL_ID,
            body=json.dumps(request_body),
            contentType="application/json",
            accept="application/json"
        )
        
        result = json.loads(response['body'].read())
        
        # Extract text embedding from response
        if 'data' in result and len(result['data']) > 0:
            embedding = result['data'][0].get('embedding', [])
            print(f"✓ Generated embedding with {len(embedding)} dimensions")
            return embedding
        else:
            print(f"✗ No embedding found in response")
            return []
    
    except Exception as e:
        print(f"✗ Error generating embedding: {e}")
        import traceback
        traceback.print_exc()
        return []

print("✓ Embedding generation function defined")

2025-11-12 18:09:58,392 - INFO - Found credentials in environment variables.


✓ Embedding generation function defined


## Test Query Embedding Generation

In [5]:
# Example: Generate embedding from user query and search
user_query = "Scared child being protected by adults"

print("\n" + "="*70)
print("GENERATING EMBEDDING FROM USER QUERY")
print("="*70)

# Generate embedding for the query
query_embedding = generate_text_embedding(user_query)

if query_embedding:
    print(f"\n✓ Query embedding generated successfully")
    print(f"  Query: '{user_query}'")
    print(f"  Embedding dimension: {len(query_embedding)}D")
    print(f"  First 5 values: {query_embedding}")
else:
    print(f"\n✗ Failed to generate query embedding")


GENERATING EMBEDDING FROM USER QUERY
📝 Generating embedding for query: 'Scared child being protected by adults'
✓ Generated embedding with 1024 dimensions

✓ Query embedding generated successfully
  Query: 'Scared child being protected by adults'
  Embedding dimension: 1024D
  First 5 values: [0.022705078125, -0.0169677734375, -0.01416015625, -0.0191650390625, 0.025146484375, 0.03076171875, -0.0179443359375, 0.01123046875, -0.0439453125, -0.06982421875, 0.01092529296875, -0.017822265625, -0.0030059814453125, 0.0223388671875, 0.041748046875, -0.0216064453125, -0.04248046875, -0.00836181640625, -0.0224609375, -0.01019287109375, 0.0247802734375, -0.016357421875, -0.017822265625, 0.059326171875, -0.0279541015625, 0.02001953125, 0.06298828125, -0.034423828125, -0.035400390625, 0.0267333984375, 0.05859375, -0.0751953125, 0.0615234375, -0.0439453125, 0.004852294921875, 0.0162353515625, 0.0247802734375, -0.0098876953125, 0.0108642578125, -0.036376953125, -0.0228271484375, -0.00078582763671875

## Search Using Generated Query Embedding

In [27]:
# Search using the generated query embedding
if query_embedding:
    print("\n" + "="*70)
    print("SEARCHING WITH GENERATED QUERY EMBEDDING")
    print("="*70)
    
    search_results_3  = query_nested_embeddings(
        query_vector=query_embedding,
        k=5
    )
    
    for hit in search_results_3["hits"]["hits"]:
        print(f"Video ID: {hit['_source']['videoName']}")
        print(f"Title: {hit['_source']['title']}")
        print(f"Score: {hit['_score']}")
        print(f"Duration: {hit['_source']['durationSec']:.2f} seconds")
        print("Matching Segment:")
        for segment in hit["inner_hits"]["embeddings"]["hits"]["hits"]:
            print(f"  Segment: {segment['_nested']['offset']}")
            print(f"    Score: {segment['_score']}")
            print(
                f"    Embedding type: {segment['fields']['embeddings.embeddingOption'][0]}"
            )
            print(f"    Start: {segment['fields']['embeddings.startSec'][0]} seconds")
            print(f"    End: {segment['fields']['embeddings.endSec'][0]} seconds")
        print("\r")


SEARCHING WITH GENERATED QUERY EMBEDDING


2025-11-12 18:45:40,227 - INFO - POST https://search-condenast-aos-domain-3hmon7me6ct3p5e46snecxe6f4.us-east-1.es.amazonaws.com:443/video_clips_nested/_search [status:200 request:1.088s]


Video ID: Jurassic World Rebirth   Official Trailer 2.mp4


KeyError: 'title'

## Index Statistics

In [7]:
# Get index statistics
try:
    stats = opensearch_client.indices.stats(index='video_clips_nested')
    doc_count = stats['indices']['video_clips_nested']['primaries']['docs']['count']
    store_size = stats['indices']['video_clips_nested']['primaries']['store']['size_in_bytes']
    
    print("\n" + "="*70)
    print("INDEX STATISTICS")
    print("="*70)
    print(f"Index: video_clips_nested")
    print(f"Total documents: {doc_count}")
    print(f"Store size: {store_size / (1024*1024):.2f} MB")
    print()
except Exception as e:
    print(f"Error fetching statistics: {e}")

2025-11-12 18:10:05,790 - INFO - GET https://search-condenast-aos-domain-3hmon7me6ct3p5e46snecxe6f4.us-east-1.es.amazonaws.com:443/video_clips_nested/_stats [status:200 request:0.284s]



INDEX STATISTICS
Index: video_clips_nested
Total documents: 392
Store size: 3.20 MB

